In [1]:
# first round rmse: 0.0281364

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv('societe_generale_training.csv',parse_dates=['time_stamp'])
test = pd.read_csv('societe_generale_test.csv',parse_dates=['time_stamp'])
submission = pd.read_csv('societe_generale_sample_submission.csv')

In [4]:
test.head()

,serial_number,time_stamp,1 year,2 year,3 year,4 year,6 year,7 year,8 year,9 year,10 year,15 year,20 year,25 year,30 year
0,1,2019-01-15,-0.160,-0.153,-0.154,-0.149,-0.146,-0.146,-0.105,-0.045,0.014,0.269,0.486,0.605,0.696
1,2,2019-01-11,-0.148,-0.153,-0.157,-0.155,-0.152,-0.150,-0.107,-0.046,0.014,0.269,0.488,0.605,0.696
2,3,2019-01-10,-0.147,-0.148,-0.154,-0.155,-0.146,-0.146,-0.102,-0.036,0.024,0.279,0.505,0.621,0.710
3,4,2019-01-09,-0.144,-0.147,-0.154,-0.152,-0.141,-0.136,-0.092,-0.033,0.029,0.286,0.519,0.631,0.720
4,5,2019-01-08,-0.146,-0.147,-0.158,-0.160,-0.152,-0.146,-0.102,-0.049,0.014,0.269,0.500,0.614,0.706


In [5]:
data = data.fillna(method='ffill')
test = test.fillna(method='ffill')

In [6]:
data.shape

(8430, 16)

In [7]:
X = data.iloc[:,[2,3,4,5,7,8,9,10,11,12,13,14,15]]
y = data.iloc[:,6].values

In [8]:
X_test = test.iloc[:,2:]

In [9]:
X_train, X_valid , y_train, y_valid = train_test_split(X,y,test_size=0.2)

In [10]:
lgbm_train = lgb.Dataset(X_train,y_train)
lgbm_valid = lgb.Dataset(X_valid,y_valid,reference=lgbm_train)

In [11]:
params = {'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'rmse',
          'bagging_freq':5,
          'feature_fraction':0.9,
          'max_depth':100,
          'num_leaves':450,
          'learning_rate':0.3
          
         }

In [12]:
gbm = lgb.train(
                params,
               lgbm_train,
               valid_sets=lgbm_valid,
                num_boost_round=2000,
                early_stopping_rounds=10
                #verbose_eval=False # verbose = 0
               )

[1]	valid_0's rmse: 2.0726
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 1.45108
[3]	valid_0's rmse: 1.01591
[4]	valid_0's rmse: 0.711235
[5]	valid_0's rmse: 0.498469
[6]	valid_0's rmse: 0.349716
[7]	valid_0's rmse: 0.245718
[8]	valid_0's rmse: 0.173555
[9]	valid_0's rmse: 0.123705
[10]	valid_0's rmse: 0.089611
[11]	valid_0's rmse: 0.0668045
[12]	valid_0's rmse: 0.0520029
[13]	valid_0's rmse: 0.0429645
[14]	valid_0's rmse: 0.037635
[15]	valid_0's rmse: 0.0347205
[16]	valid_0's rmse: 0.033044
[17]	valid_0's rmse: 0.0321422
[18]	valid_0's rmse: 0.0316025
[19]	valid_0's rmse: 0.0311122
[20]	valid_0's rmse: 0.0307261
[21]	valid_0's rmse: 0.0305544
[22]	valid_0's rmse: 0.0303496
[23]	valid_0's rmse: 0.0303033
[24]	valid_0's rmse: 0.0300179
[25]	valid_0's rmse: 0.0298104
[26]	valid_0's rmse: 0.0297566
[27]	valid_0's rmse: 0.0296479
[28]	valid_0's rmse: 0.0294908
[29]	valid_0's rmse: 0.0294155
[30]	valid_0's rmse: 0.0293213
[31]	valid_0's rmse: 0.029141
[32

In [13]:
GS = False

In [14]:
if GS:
    grid_params = {#'max_depth' : [100,200],
                   #'num_leaves' : [450,460,470],
                    #'learning_rate': [0.3,0.31,0.25]
                    #'feature_fraction': [0.9,1],
                    #'bagging_fraction': [1],
                    #'bagging_freq' : [5,6]
    }
    mdl = lgb.LGBMRegressor(boosting_type= params['boosting_type'], # these will be commented when the parameter is in grid_params, uncomment when you get the best results.
                           objective = params['objective'],
                            metric = params['metric'],
                            #num_leaves = params['num_leaves'], 
                            #learning_rate = params['learning_rate'],
                            #feature_fraction = params['feature_fraction'],
                            #bagging_fraction = params['bagging_fraction'],
                            #bagging_freq = params['bagging_freq']
                           )
    grid = GridSearchCV(mdl,
                       grid_params,
                        cv = 5,
                        scoring = 'neg_mean_squared_error'
                       )
    grid.fit(X_train,y_train)
    print(grid.best_params_)

In [15]:
pred_buff = []
n_itr = 5
for i in range(n_itr):
    X_train, X_valid , y_train, y_valid = train_test_split(X,y,test_size=0.2,random_state=i) # random state is the number in the iteration
    lgbm_train = lgb.Dataset(X_train,y_train) # Preparing data for lgb
    lgbm_valid = lgb.Dataset(X_valid,y_valid,reference=lgbm_train) # Preparing data for lgb
    
    gbm_itr = lgb.train(
                params,
               lgbm_train,
               valid_sets=lgbm_valid,
                num_boost_round=2000,
                early_stopping_rounds=50, # stop if the results do not improve in 5 rounds
                verbose_eval=False # verbose = 0
               )
    
    y_pred = gbm.predict(X_test,
                     num_iteration=gbm.best_iteration) # run the number of iterations for the best number
    pred_buff.append(y_pred)

In [16]:
submit = True

In [17]:
if submit:
    submission['5 year'] = y_pred
    submission.to_csv('submission.csv',index=False)